## Reference Links - 

* https://jalammar.github.io/illustrated-word2vec/
* https://www.analyticsvidhya.com/blog/2019/07/how-to-build-recommendation-system-word2vec-python/
* https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial
* https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5
* https://bio.nlplab.org/
* https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
* https://machinelearningmastery.com/develop-word-embeddings-python-gensim/


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

pd.options.display.max_colwidth = 200

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

As we can not feed words directly into most of the models, we need numeric values for each text. Earlier we used vectorization techniques like - Tfifd and CountVectorize to represent each text. In this notebook we learn about word and sentence embeddings are how they can be used for downstream tasks like text classification.

## Table of Contents

* Embeddings
    * Word2Vec
    * FastText
    * Doc2Vec
* Visualizing Representations
* Evaluating Representations on Text Classification with Logistic Regression

In [ ]:
df = pd.read_csv('/kaggle/input/nlp-specialization-data/Cleaned_POS_Medical_Notes.csv') #for excel file use read_excel

In [ ]:
df.head(3)

<a id='emb'></a>

### Embeddings

Word embedding is one of the most important techniques in natural language processing(NLP), where words are mapped to vectors of real numbers. Word embedding is capable of capturing the meaning of a word in a document, semantic and syntactic similarity, relation with other words. It also has been widely used for recommender systems and text classification. This tutorial will show a brief introduction of genism word2vec model with an example of generating word embedding for review texts.

Embedding techniques have been used on different entities - character, word, document, paragraph etc. Apart from word embeddings, we also discuss document embedding in this section.

<a id='w2v'></a>

#### Word2Vec

Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and human might cluster in one cluster. Another interesting property of word2vec is, it preserves distance between similar items.

<img src=https://cdn.analyticsvidhya.com/wp-content/uploads/2017/06/06062705/Word-Vectors.png>

Let us build Word2Vec embeddings using our own dataset.

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
input_word2vec = df.clean_text.tolist()
input_word2vec = [i.split() for i in input_word2vec]

#phrases = Phrases(input_word2vec, min_count=30, progress_per=10000)
#bigram = Phraser(phrases)
#input_word2vec = bigram[input_word2vec]

In [ ]:
print (len(input_word2vec))
print (input_word2vec[0][:20])

Gensim word2vec requires texts in the above format for training.

In [ ]:
model = Word2Vec(min_count=3)
model.build_vocab(input_word2vec)
model.train(input_word2vec,total_examples = model.corpus_count,epochs=10)

In [ ]:
print (model.wv['diabetes'].shape)
model.wv['diabetes']

In [ ]:
print (len(model.wv.key_to_index))

So, using W2V we are able to calculate 100 dimensional embeddings for each of 6305 words. Let us check few similar word pairs.

In [ ]:
model.wv.most_similar('diabetes')[:5]

In [ ]:
model.wv.most_similar('bladder')[:5]

In [ ]:
model.wv.most_similar('gait')[:5]

Hence, hypertension is very close to diabetes. Similarly, bowel is very similar to the word bladder.

In [ ]:
def get_document_vector(x,model,vector_size=100):
    vec = np.zeros(vector_size)
    count = 0
    for i in x.split():
        try:
            vec += model[i]
            count += 1
        except:
            pass
    if count > 0:
        vec /= count
    
    return vec

In [ ]:
word2vec_features = np.array([get_document_vector(i,model) for i in df.clean_text.tolist()])
print (word2vec_features.shape)

We calculate average embedding score for each of the texts.

In [ ]:
model.wv.save_word2vec_format("w2v.txt",binary=False)

<a id='ft'></a>

#### FastText

FastText is a technique developed by Facebook research for sub-word level representation learning. Fasttext is very fast and efficient for out of vocabulary word and sub-word level embeddings.

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
model = FastText(min_count=3,vector_size=100)
model.build_vocab(input_word2vec)
model.train(input_word2vec,total_examples = model.corpus_count,epochs=20)

In [ ]:
model.wv.most_similar('diabetes')[:5]

In [ ]:
model.wv.most_similar('bladder')[:5]

In [ ]:
model.wv.most_similar('gait')[:5]

In [ ]:
print (len(model.wv.key_to_index))

In [ ]:
fasttext_features = np.array([get_document_vector(i,model) for i in df.clean_text.tolist()])
print (fasttext_features.shape)

In [ ]:
#model.wv.save_word2vec_format("ft.txt",binary=False)

#### Pubmed pretrained embeddings

This embedding was trained on huge biomedical corpus. We load the trained model and do some exploration. The bin file can be downloaded from https://bio.nlplab.org/

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
pubmed_model = KeyedVectors.load_word2vec_format('/kaggle/input/nlp-specialization-data/pubmed2018_w2v_200D/pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin',binary=True)

In [ ]:
pubmed_model.most_similar('diabetes')[:5]

In [ ]:
pubmed_model.most_similar('bladder')[:5]

In [ ]:
pubmed_model.most_similar('gait')[:5]

In [ ]:
print (len(pubmed_model.index_to_key))

In [ ]:
pubmed_model['gait'].shape

In the pubmed pretrained model, we have 200 dimensional embeddings for 2.6 million words, mostly from biomedical domain.

In [ ]:
pubmed_features = np.array([get_document_vector(i,pubmed_model,vector_size=200) for i in df.clean_text.tolist()])
print (pubmed_features.shape)

In [ ]:
# pubmed_model.save_word2vec_format('pubmed_wv.txt',binary=False)

<a id='d2v'></a>

#### Doc2Vec

After understanding what word2vec is, it will be easier to understand how doc2vec works.

As said, the goal of doc2vec is to create a numeric representation of a document, regardless of its length. But unlike words, documents do not come in logical structures such as words, so the another method has to be found.

<img src=https://miro.medium.com/max/608/0*x-gtU4UlO8FAsRvL.>

Using Gensim, we can learn Doc2Vec representation as well for our dataset.

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df.clean_text.tolist())]
model = Doc2Vec(vector_size=100)
model.build_vocab(documents)
model.train(documents,total_examples = model.corpus_count,epochs=10)

In [ ]:
model.docvecs[0]

In [ ]:
model.docvecs.most_similar(0)[:5]

Under Doc2Vec, the 1st text is most similar to the 340th text.

In [ ]:
doc2vec_features = np.array([model.docvecs[i] for i in range(len(df))])

<a id='viz'></a>

### Visualizing Representations

We can use quantitative methods for evaluating representations learned from different model. However, as the task is unsupervised, most of the evaluations are based on similarity based metrics. As we see below, some of the representations preserve local properties i.e - similar texts have similar representation and cluster together. On the other hand, some methods look for global structures. Usually, people evaluate representations using downstream task specific metrics.

As we can not visualize 100-dim vectors, we use t-SNE embeddings to reduce the dimensions into 2. t-Distributed Stochastic Neighbor Embedding (t-SNE) is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional dfsets.

In [ ]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word2vec_features)

In [ ]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_ft = tsne_model.fit_transform(fasttext_features)

In [ ]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_pm = tsne_model.fit_transform(pubmed_features)

In [ ]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_d2v = tsne_model.fit_transform(doc2vec_features)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import HTML, Image
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px
#init_notebook_mode(connected=True)

all_colors = px.colors.qualitative.Plotly
print (all_colors)

In [ ]:
vis_df = pd.DataFrame()
vis_df['x'] = tsne_w2v[:,0]
vis_df['y'] = tsne_w2v[:,1]
vis_df['speciality'] = df.label

fig = px.scatter(vis_df, x="x", y="y", color=vis_df.speciality, title="Projection of Clinical Texts (based on Word2Vec)", height=600, width=1000) #hover_df=['top_words']
fig.show()

In [ ]:
vis_df = pd.DataFrame()
vis_df['x'] = tsne_ft[:,0]
vis_df['y'] = tsne_ft[:,1]
vis_df['speciality'] = df.label

fig = px.scatter(vis_df, x="x", y="y", color=vis_df.speciality, title="Projection of Clinical Texts (based on FastText)", height=600, width=1000) #hover_df=['top_words']
fig.show()

In [ ]:
vis_df = pd.DataFrame()
vis_df['x'] = tsne_pm[:,0]
vis_df['y'] = tsne_pm[:,1]
vis_df['speciality'] = df.label

fig = px.scatter(vis_df, x="x", y="y", color=vis_df.speciality, title="Projection of Clinical Texts (based on PubMed)", height=600, width=1000) #hover_df=['top_words']
fig.show()

In [ ]:
vis_df = pd.DataFrame()
vis_df['x'] = tsne_d2v[:,0]
vis_df['y'] = tsne_d2v[:,1]
vis_df['speciality'] = df.label

fig = px.scatter(vis_df, x="x", y="y", color=vis_df.speciality, title="Projection of Clinical Texts (based on Doc2Vec)", height=600, width=1000) #hover_df=['top_words']
fig.show()

### Using Embeddings with Logistic Regression

In this section we fit simple logistic regression with embeddings as features to classify the clinial notes into the specialities.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
lg = LogisticRegression(multi_class='auto',solver='lbfgs')
cv_scores = cross_val_score(X=doc2vec_features,y=df.label,cv=StratifiedKFold(n_splits=5,random_state=42,shuffle=True),estimator=lg)
print (cv_scores, np.mean(cv_scores))

In [ ]:
lg = LogisticRegression(multi_class='auto',solver='lbfgs')
cv_scores = cross_val_score(X=word2vec_features,y=df.label,cv=StratifiedKFold(n_splits=5,random_state=42,shuffle=True),estimator=lg)
print (cv_scores, np.mean(cv_scores))

In [ ]:
lg = LogisticRegression(multi_class='auto',solver='lbfgs')
cv_scores = cross_val_score(X=fasttext_features,y=df.label,cv=StratifiedKFold(n_splits=5,random_state=42,shuffle=True),estimator=lg)
print (cv_scores, np.mean(cv_scores))

In [ ]:
lg = LogisticRegression(multi_class='auto',solver='lbfgs')
cv_scores = cross_val_score(X=pubmed_features,y=df.label,cv=StratifiedKFold(n_splits=5,random_state=42,shuffle=True),estimator=lg)
print (cv_scores, np.mean(cv_scores))

Using FastText and PubMed embeddings as features, we can achieve good performance using a simple logistic regression model.